In [ ]:
import torch
print(torch.cuda.is_available())

print(torch.cuda.device_count())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
phase = 1

import numpy as np
import random

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # For multi-GPU setups, if any
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Example usage
set_seed(58)  # Replace 42 with your chosen seed


In [ ]:
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler

def normalizaData(df):
    # Convert DataFrame to PyTorch Tensor and send to GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Normalizing Angular Data (Robotic Joints)
    rad_cols = ['Tool_rx', 'Tool_ry', 'Tool_rz', 'Joint_base', 'Joint_Shoulder', 'Joint_Elbow', 'Joint_W1', 'Joint_W2', 'Joint_W3']
    for col in rad_cols:
        # Ensure the data is in the form of a PyTorch Tensor
        rad_tensor = torch.tensor(df[col].values, device=device, dtype=torch.float32)
        df[col + '_cos'] = torch.cos(rad_tensor).cpu().numpy()  # Compute cosine and convert back to NumPy for DataFrame
        df[col + '_sin'] = torch.sin(rad_tensor).cpu().numpy()  # Compute sine and convert back to NumPy for DataFrame
    
    # Normalizing Linear Data (Speed, Acceleration, Energy Consumed, and Time)
    # Min-Max Scaling: Scales the data to a fixed range, typically 0 to 1
    scaler = MinMaxScaler()
    linear_cols = ['Speed', 'Acceleration', 'Time', 'Energy_Consumped']
    df[linear_cols] = scaler.fit_transform(df[linear_cols])
    
    return df, scaler


In [ ]:
import pandas as pd
import torch

# Load data
real_data = pd.read_csv('phase RLHF dataset/remaining_paths.csv')


# Fix a typo in the 'point_type' column
real_data['point_type'] = real_data['point_type'].replace('approuch', 'approach')

# Map 'point_type' and 'Movement Type' to numeric values
points = {'pick': 1, 'approach': 2, 'start_point': 3, 'box1': 4, 'way_point': 5}
movement_types = {'MoveL': 1, 'MoveJ': 2, 'Movel': 1, 'Movej': 2}
real_data['point_type'] = real_data['point_type'].map(points)
real_data['Movement Type'] = real_data['Movement Type'].map(movement_types)

# Drop columns 'id' and 'row_index'
real_data.drop(['id'], axis=1, inplace=True)

# Assuming here that the use of PyTorch would be to handle numeric data for neural network processing
# Convert DataFrame to PyTorch Tensor and send to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tensor_data = torch.tensor(real_data.values, dtype=torch.float, device=device)

# Print number of columns in the DataFrame
print(len(real_data.columns))


In [ ]:
# Normalize the data
df_normalized, scaler = normalizaData(real_data.copy())

# Define the features to be used
features = [
    'point_type', 'Tool_x', 'Tool_y', 'Tool_z', 'Tool_rx_cos', 'Tool_rx_sin', 'Tool_ry_cos',
    'Tool_ry_sin', 'Tool_rz_cos', 'Tool_rz_sin', 'Joint_base_cos',
    'Joint_base_sin', 'Joint_Shoulder_cos', 'Joint_Shoulder_sin',
    'Joint_Elbow_cos', 'Joint_Elbow_sin', 'Joint_W1_cos', 'Joint_W1_sin',
    'Joint_W2_cos', 'Joint_W2_sin', 'Joint_W3_cos', 'Joint_W3_sin', 'Speed', 'Acceleration', 'Movement Type', 'Time', 'Energy_Consumped'
]

# Calculate the number of features
features_num = len(features)

# Select the specified features from the normalized DataFrame
df_normalized = df_normalized[features]

# Display the first few rows of the selected features
df_normalized.head()


In [ ]:
import numpy as np
import pandas as pd

# Assuming df_normalized has been defined and features_num has been calculated as before
# Add a 'path_id' column to help in grouping. Creating a dummy 'path_id' that repeats every 5 rows
num_paths = len(df_normalized) // 5  # Use integer division
path_id = np.repeat(np.arange(num_paths), 5)
df_normalized['path_id'] = path_id

# Ensure the length of path_id matches the number of rows in df_normalized
if len(path_id) < len(df_normalized):
    extra_ids = np.array([num_paths] * (len(df_normalized) - len(path_id)))
    path_id = np.concatenate([path_id, extra_ids])

# Now, sort and group by 'path_id' and reshape
grouped = df_normalized.groupby('path_id').apply(lambda x: x.iloc[:, :-1].values.reshape(1, 5, features_num))

# Drop the 'path_id' column after reshaping
df_normalized.drop(['path_id'], axis=1, inplace=True)

# Concatenate the groups back into a single numpy array
reshaped_data = np.concatenate(grouped.values, axis=0)

# Save the reshaped data to a file. Assume 'phase' is defined elsewhere
np.save(f"real_normalized_dataset_shape_3D_ph{phase} WGAN.npy", reshaped_data)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# Setting the aesthetic style of the plots
sns.set_style('whitegrid')

col_16 = real_data['Time']
summed_time = col_16.groupby(real_data.index // 5).sum()
plt.figure(figsize=(10, 6))
sns.histplot(summed_time, kde=True, bins=10 , label = "Time")
plt.title('Real Data - Distribution of Cycle time')
plt.xlabel('Time (seconds)')
plt.ylabel('Frequency')
plt.legend()
plt.savefig("Real Data - Distribution of Cycle time wgan")
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# Setting the aesthetic style of the plots
sns.set_style('whitegrid')


col_17 = real_data['Energy_Consumped']
summed_energy_consumed = col_17.groupby(real_data.index // 5).sum()
plt.figure(figsize=(10, 6))

sns.histplot(summed_energy_consumed, kde=True, bins=10, label="Energy_Consumped")
plt.title('Real Data - Distribution of Energy Consumed')
plt.xlabel('Energy Consumed')
plt.ylabel('Frequency')
plt.legend()
plt.savefig("Real Data - Distribution of Energy Consumed wgan.pdf")
plt.show()

In [ ]:


'''class Generator(nn.Module):
    def __init__(self, noise_dim, num_features, sequence_length=5):
        super(Generator, self).__init__()
        self.noise_dim = noise_dim
        self.num_features = num_features
        self.sequence_length = sequence_length

        self.fc_initial = nn.Sequential(
            nn.Linear(self.noise_dim, 5 * self.num_features),
            nn.LeakyReLU(0.2)
        )
        # Define LSTM layers
        self.lstm1 = nn.LSTM(self.num_features, 1024, batch_first=True)
        self.lstm2 = nn.LSTM(1024, 256, batch_first=True)
        self.lstm3 = nn.LSTM(256, 128, batch_first=True)
        self.lstm4 = nn.LSTM(128, 64, batch_first=True)
        self.lstm5 = nn.LSTM(64, 64, batch_first=True)

        # Final fully connected layer
        self.fc = nn.Linear(64, self.num_features)
        self.relu = nn.ReLU()  # ReLU activation layer to use after LSTMs
        self.tanh = nn.Tanh()  # Tanh activation for the final output

    def forward(self, noise):
        x = self.fc_initial(noise)
        x = x.view(-1, self.sequence_length, self.num_features)
        # Passing through each LSTM layer followed by ReLU activation
        x, _ = self.lstm1(x)
        x = self.relu(x)
        x, _ = self.lstm2(x)
        x = self.relu(x)
        x, _ = self.lstm3(x)
        x = self.relu(x)
        x, _ = self.lstm4(x)
        x = self.relu(x)
        x, _ = self.lstm5(x)
        x = self.relu(x)
        
        # Passing the final LSTM output through the fully connected layer followed by Tanh
        x = x.contiguous().view(-1, 64)  # Flatten the output for the fully connected layer
        print(x.shape, "flatten")
        x = self.fc(x)
        x = self.tanh(x)  # Applying Tanh after final FC

        x = x.view(-1, self.sequence_length, self.num_features)
        return x
'''


'''class TimeDistributed(nn.Module):
    def __init__(self, module, batch_first=False):
        super(TimeDistributed, self).__init__()
        self.module = module
        self.batch_first = batch_first

    def forward(self, x):
        if len(x.size()) <= 2:
            return self.module(x)
        # Squash samples and timesteps into a single axis
        x_reshape = x.contiguous().view(-1, x.size(-1))  # (samples * timesteps, input_size)
        y = self.module(x_reshape)
        # We have to reshape Y
        if self.batch_first:
            y = y.contiguous().view(x.size(0), -1, y.size(-1))  # (samples, timesteps, output_size)
        else:
            y = y.view(-1, x.size(1), y.size(-1))  # (timesteps, samples, output_size)
        return y

class Generator(nn.Module):
    def __init__(self, noise_dim, num_features, sequence_length=5):
        super(Generator, self).__init__()
        self.noise_dim = noise_dim
        self.num_features = num_features
        self.sequence_length = sequence_length

        self.fc_initial = nn.Sequential(
            nn.Linear(self.noise_dim, 5 * self.num_features),
            nn.LeakyReLU(0.2)
        )
        # Define LSTM layers
        self.lstm1 = nn.LSTM(self.num_features, 1024, batch_first=True)
        self.lstm2 = nn.LSTM(1024, 256, batch_first=True)
        self.lstm3 = nn.LSTM(256, 128, batch_first=True)
        self.lstm4 = nn.LSTM(128, 64, batch_first=True)
        self.lstm5 = nn.LSTM(64, 64, batch_first=True)

        # TimeDistributed fully connected layer
        self.fc_final = TimeDistributed(nn.Linear(64, self.num_features), batch_first=True)

        self.tanh = nn.Tanh()

    def forward(self, noise):
        x = self.fc_initial(noise)
        x = x.view(-1, self.sequence_length, self.num_features)

        # Passing through each LSTM layer
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x, _ = self.lstm4(x)
        x, _ = self.lstm5(x)
        
        # Passing the final LSTM output through the TimeDistributed fully connected layer
        x = self.fc_final(x)
        x = self.tanh(x)  # Applying Tanh after final FC

        return x

# Example of initializing the Generator
noise_dim = 100
num_features = 27
generator = Generator(noise_dim, num_features)
noise = torch.randn(33, noise_dim)  # Example noise batch
generated_data = generator(noise)
print(generated_data.shape)  # Expected shape: [33, 5, 27]
'''
'''
class Generator(nn.Module):
    """An LSTM based generator. It expects a sequence of noise vectors as input.

    Args:
        in_dim: Input noise dimensionality
        out_dim: Output dimensionality
        n_layers: number of lstm layers
        hidden_dim: dimensionality of the hidden layer of lstms

    Input: noise of shape (batch_size, seq_len, in_dim)
    Output: sequence of shape (batch_size, seq_len, out_dim)
    """

    def __init__(self, in_dim, out_dim, n_layers=5, hidden_dim=256):
        super().__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.out_dim = out_dim

        self.lstm = nn.LSTM(in_dim, hidden_dim, n_layers, batch_first=True)
        self.linear = nn.Sequential(nn.Linear(hidden_dim, out_dim), nn.Tanh())
        self.relu = nn.ReLU()  # ReLU activation layer to use after LSTMs

    def forward(self, input):
        batch_size, seq_len = input.size(0), input.size(1)
        h_0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        c_0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim)

        recurrent_features, _ = self.lstm(input, (h_0, c_0))
        outputs = self.linear(recurrent_features.contiguous().view(batch_size*seq_len, self.hidden_dim))
        outputs = outputs.view(batch_size, seq_len, self.out_dim)'''




In [ ]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, noise_dim, num_features, sequence_length=5):
        super(Generator, self).__init__()
        self.noise_dim = noise_dim
        self.num_features = num_features
        self.sequence_length = sequence_length

        self.fc_initial = nn.Sequential(
           # nn.Linear(self.noise_dim, 5 * self.num_features),
            nn.Linear(self.noise_dim, 5 * self.num_features),
            nn.LeakyReLU(0.2)
        )
        # Define LSTM layers
        self.lstm1 = nn.LSTM(self.num_features, 1024, batch_first=True)
        self.lstm2 = nn.LSTM(1024, 256, batch_first=True)
        self.lstm3 = nn.LSTM(256, 128, batch_first=True)
        self.lstm4 = nn.LSTM(128, 64, batch_first=True)
        self.lstm5 = nn.LSTM(64, 64, batch_first=True)

        self.final_layer = nn.Sequential(
            nn.Linear(64, self.num_features),
            nn.Tanh()
        )
        self.tanh = nn.Tanh()  # Tanh activation for the final output
        self.relu = nn.ReLU()

    def forward(self, noise):
        x = self.fc_initial(noise)
        x = x.view(-1, self.sequence_length, self.num_features)
        # Passing through each LSTM layer followed by ReLU activation
        x, _ = self.lstm1(x)
        x = self.relu(x)
        x, _ = self.lstm2(x)
        x = self.relu(x)
        x, _ = self.lstm3(x)
        x = self.relu(x)
        x, _ = self.lstm4(x)
        x = self.relu(x)
        x, _ = self.lstm5(x)
        x = self.relu(x)
         # Apply final transformation to each time point
        x = self.final_layer(x)
        return x
        


In [ ]:

import torch.nn.functional as F

def sse_module(x):
    """
    Spatial Squeeze and Excitation (SSE) module function for PyTorch.
    
    Parameters:
    - x (torch.Tensor): The input tensor with dimensions (batch_size, channels, length).

    Returns:
    - torch.Tensor: Output tensor after applying the SSE attention mechanism.
    """
    # Ensure the input tensor is on the appropriate device (GPU if available)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    x = x.to(device)

    # Convolution layer to expand feature dimensions
    conv = nn.Conv1d(in_channels=x.size(1), out_channels=64, kernel_size=1, padding='same').to(device)
    x = conv(x)
    x = F.relu(x)

    # Squeeze step: Global Average Pooling
    squeeze = F.adaptive_avg_pool1d(x, 1).squeeze(-1)  # Global average pooling and removing the last dimension
    
    # Fully connected layer to generate attention weights
    fc = nn.Linear(64, 64).to(device)
    excitation = fc(squeeze)
    excitation = torch.sigmoid(excitation)
    
    # Expand dimensions to match convolution output for multiplication
    excitation = excitation.unsqueeze(-1)
    
    # Excitation step: Apply attention weights
    out = x * excitation  # Element-wise multiplication

    return out

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, features_num):
        super(Discriminator, self).__init__()
        self.features_num = features_num
        self.input_shape = (5, features_num)  # Shape of input data
        self.lstm_input_size = features_num 
        # LSTM layers
        self.lstm1 = nn.LSTM(self.lstm_input_size, 1024, batch_first=True)
        self.lstm2 = nn.LSTM(1024, 256, batch_first=True)
        self.lstm3 = nn.LSTM(256, 128, batch_first=True)
        self.lstm4 = nn.LSTM(128, 32, batch_first=True)
        self.lstm5 = nn.LSTM(32, 32, batch_first=True)
        self.lstm6 = nn.LSTM(32, 32, batch_first=True)

        # Dropout layer
        self.dropout = nn.Dropout(0.3)

        # Fully connected layer
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        # Reshape condition to match input data for concatenation
        # Concatenate input data and condition
        merged_input = torch.cat([x], dim=2)

        # Apply SSE module
        #print(merged_input.shape, "merged_input before")
        #merged_input = merged_input.permute(0, 1, 2)  # Change shape to (batch, channels, length)
        #print(merged_input.shape, "merged_input")
       # (None, 5, 29) merged_input

        #sse_out = sse_module(merged_input)
        #sse_out = sse_out.permute(0, 2, 1)  # Change back to (batch, length, channels)
        #print(sse_out.shape,"sse_out")
      
        x, _ = self.lstm1(merged_input)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x = self.dropout(x)
        x, _ = self.lstm4(x)
        x = self.dropout(x)
        x, _ = self.lstm5(x)
        x, _ = self.lstm6(x)

        # Take the output from the last LSTM step
        x = x[:, -1, :]

        # Apply fully connected layer
        x = self.fc(x)
        return x

In [ ]:
def denormalize_min_max(data_normalized, col_index ,col_index_realdata):
    """
    Denormalize data normalized using Min-Max scaling.

    Parameters:
    - data_normalized (torch.Tensor): Normalized data tensor with shape (batch_size, seq_len, num_features).
    - real_data_tensor (torch.Tensor): Original real data tensor used to determine min and max values.
    - col_index (int): Index of the column to denormalize in the normalized data.
    - col_index_realdata (int): Index of the corresponding column in the real data.

    Returns:
    - torch.Tensor: Denormalized column of data.
    
    """
    device = data_normalized.device  # Ensure the device is consistent
    real_data_tensor = torch.tensor(real_data.to_numpy(), dtype=torch.float32).to(device)
    # Calculate min and max values from the real data
    min_vals = torch.min(real_data_tensor, dim=0)[0].to(device)  # Shape: (num_features,)
    max_vals = torch.max(real_data_tensor, dim=0)[0].to(device)  # Shape: (num_features,)

    # Extract the relevant min and max values for broadcasting
    min_val = min_vals[col_index_realdata].unsqueeze(0).unsqueeze(0)  # Shape: (1, 1)
    max_val = max_vals[col_index_realdata].unsqueeze(0).unsqueeze(0)  # Shape: (1, 1)

    # Apply the denormalization formula
    col_denormalized = data_normalized[:, :, col_index] * (max_val - min_val) + min_val

    return col_denormalized

In [ ]:
def denormalize_data(arr, scaler, features):
    
    """
    Denormalizes the data from a normalized array using the provided scaler.
    
    Parameters:
    - arr (numpy.ndarray): The normalized array to be denormalized. Expected shape: (samples, features, timesteps).
    - scaler (MinMaxScaler or StandardScaler): The scaler used to normalize the data.
    - features (list): List of feature names for the DataFrame.
    
    Returns:
    - pd.DataFrame: A DataFrame with denormalized data.
    """
    
    # Reshape array into 2D (flatten the timesteps) and create DataFrame
    reshaped_array = arr.reshape(-1, arr.shape[2])
    df = pd.DataFrame(reshaped_array, columns=features)
    
    # Denormalizing Angular Data
    # Identify columns related to cosine and sine representations
    angle_columns = [col for col in df.columns if '_cos' in col or '_sin' in col]
    
    # Set to keep track of processed base columns to avoid duplicates
    processed_bases = set()
    
    for col in angle_columns:
        if '_cos' in col:
            base_col = col.replace('_cos', '')
            if base_col not in processed_bases:
                # Combine cosine and sine to compute the angle
                df[base_col] = np.arctan2(df[base_col + '_sin'], df[base_col + '_cos'])
                # Remove the original cosine and sine columns
                df.drop(columns=[base_col + '_sin', base_col + '_cos'], inplace=True)
                # Mark the base column as processed
                processed_bases.add(base_col)
    
    # Denormalizing Linear Data
    # List of columns to denormalize
    linear_columns = ['Speed', 'Acceleration', 'Time', 'Energy_Consumped']
    
    # Ensure all specified columns are in the DataFrame before attempting denormalization
    available_linear_columns = [col for col in linear_columns if col in df.columns]
    
    if available_linear_columns:
        df[available_linear_columns] = scaler.inverse_transform(df[available_linear_columns])
    return df



In [ ]:
def physics_loss(fake_data):
    """
    Calculate the physics-based loss for generated data.

    Parameters:
    - fake_data (torch.Tensor): Generated data with shape (batch_size, seq_len, num_features).

    Returns:
    - torch.Tensor: Total physics loss.
    """
    fake_data_numpy = fake_data.detach().cpu().numpy()

    df = denormalize_data(fake_data_numpy , scaler, features)
    '''Index(['point_type', 'Tool_x', 'Tool_y', 'Tool_z', 'Speed', 'Acceleration',
       'Movement Type', 'Time', 'Energy_Consumped', 'Tool_rx', 'Tool_ry',
       'Tool_rz', 'Joint_base', 'Joint_Shoulder', 'Joint_Elbow', 'Joint_W1',
       'Joint_W2', 'Joint_W3'],
      dtype='object') '''
    #Penalty for negative values in specific columns

    #Movement type 1-> moveL , 2-> moveJ 
    #Tool x,y,z # Move l ,speed 0-3000 mm/s -> 0-3 m/s
    #Tool x,y,z # Move l ,acceleration 0-150000 mm/s2 - 0-150 m/s 
    #Denormalize speed , 13 index in real data , 22 speed in normalized data
    #[-25.2721, -25.7903, -25.5148, -25.1693, -24.9287],

    # Speed constraints for MoveL
    # i assume movment type = 1 if less than 0.5 
    filtered_speed = len(df[(df['Speed'] >= 0) & (df['Speed'] <= 3) & (df['Movement Type'] <= 1)])
    filtered_Acceleration = len(df[(df['Acceleration'] >= 0) & (df['Acceleration'] <= 150) & (df['Movement Type'] <= 1)])
    speed_loss_l = filtered_speed
    acc_loss_l = filtered_Acceleration
  

    # Speed constraints for MoveJ
        #rx,ry,rz,joints 
        #Move J speed 0-180 degree per second -> radian 3.14159
        #Move J acceleration 0 - 2292  degree/second ^2 -> radian 40
    filtered_speed_mj = len(df[(df['Speed'] >= 0) & (df['Speed'] <=  3.14159) & (df['Movement Type'] >=1)])
    filtered_Acceleration_mj = len(df[(df['Acceleration'] >= 0) & (df['Acceleration'] <= 40) & (df['Movement Type'] >= 1)])
    speed_loss_j = filtered_speed_mj
    acc_loss_j = filtered_Acceleration_mj
  

    # Joint angle constraints
       #validate_joint_angles
        #-363 to 363 degree/s ---->    -6.33555 , 6.33555 rad
        #Joint limits as defined in the screenshot
    joint_columns = ['Joint_base', 'Joint_Shoulder', 'Joint_Elbow', 'Joint_W1', 'Joint_W2', 'Joint_W3']
    # Filter the DataFrame
    filtered_df_joints = df[(df[joint_columns] < -6.33555) | (df[joint_columns] > 6.33555)].dropna(how='all', subset=joint_columns)
    joints_loss = len(filtered_df_joints)
   
    #time, energy negative 
    filtered_df = df[(df['Time'] < 0) | (df['Energy_Consumped'] < 0)]
    time_energy_loss = len(filtered_df)
    total_loss = speed_loss_l + acc_loss_l + speed_loss_j + acc_loss_j + joints_loss + time_energy_loss
    return total_loss


In [ ]:

def gradient_penalty(discriminator, real_data, fake_data,lambda_gp=10):

    """
    Calculate the gradient penalty for WGAN-GP with CuDNN disabled.

    Parameters:
    - discriminator (nn.Module): The discriminator model.
    - real_data (torch.Tensor): Real data samples.
    - fake_data (torch.Tensor): Generated data samples.
    - device (torch.device): Device to run computations on (CPU or GPU).
    - lambda_gp (float): Gradient penalty coefficient.

    Returns:
    - torch.Tensor: Gradient penalty term.
    """
    batch_size = real_data.size(0)

    # Interpolate between real and fake data
    alpha = torch.rand(batch_size, 1, 1, device=device)
    interpolates = alpha * real_data + (1 - alpha) * fake_data
    interpolates.requires_grad_(True)

    with torch.backends.cudnn.flags(enabled=False):
        # Discriminator prediction for interpolates
        prediction = discriminator(interpolates)

        # Calculate gradients
        gradients = torch.autograd.grad(
            outputs=prediction,
            inputs=interpolates,
            grad_outputs=torch.ones_like(prediction, device=device),
            create_graph=True,
            retain_graph=True,
            only_inputs=True
        )[0]

    # Flatten the gradients to (batch_size, -1) for norm calculation
    gradients = gradients.reshape(batch_size, -1)

    # Compute the L2 norm of the gradients
    gradient_norm = gradients.norm(2, dim=1)
    gradient_penalty = lambda_gp * ((gradient_norm - 1) ** 2).mean()

    return gradient_penalty


In [ ]:

def wasserstein_loss(y_true, y_pred):
    """
    Calculate the Wasserstein loss for WGAN.

    Parameters:
    - y_true (torch.Tensor): The ground truth labels, typically -1 for real and 1 for fake data.
    - y_pred (torch.Tensor): The predictions from the discriminator.

    Returns:
    - torch.Tensor: The computed Wasserstein loss.
    """
    # Ensure that predictions and labels are on the same device (e.g., GPU)
    device = y_pred.device  # Assuming y_pred is already on the correct device
    y_true = y_true.to(device)

    # Compute the Wasserstein loss as the negative mean of products
    return -torch.mean(y_true * y_pred)

In [ ]:
import torch

def generate_noise(batch_size, noise_dim, seed=None):
    """
    Generate uniform noise for the GAN.

    Parameters:
    - batch_size (int): Number of samples.
    - noise_dim (int): Dimensionality of the noise vector.
    - seed (int, optional): Random seed for reproducibility.

    Returns:
    - torch.Tensor: Noise tensor.
    """
    '''if seed is not None:
        print("enter seed")
        torch.manual_seed(seed)'''
    
    return torch.randn(batch_size, noise_dim).to('cuda' if torch.cuda.is_available() else 'cpu')
    # Define the shape of the tensor
    #shape = (batch_size, 5, 27)
    #torch.rand(batch_size, noise_dim) 


In [ ]:

import torch.optim as optim

def setup_gan(generator, discriminator, lr=0.00005):
    """
    Setup the GAN by initializing optimizers and moving models to the appropriate device.

    Parameters:
    - generator (nn.Module): The generator model.
    - discriminator (nn.Module): The discriminator model.
    - lr (float): Learning rate for the optimizers.

    Returns:
    - Tuple of optimizers for generator and discriminator.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    generator.to(device)
    discriminator.to(device)

    optimizer_g = optim.Adam(generator.parameters(), lr=lr)
    optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0001)

    return optimizer_g, optimizer_d, device


In [ ]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

def train_batches(generator, discriminator, optimizer_g, optimizer_d, device, epochs, batch_size, noise_dim,lambda_gp=20):
    """
    Train the GAN using Wasserstein loss with gradient penalty.

    Parameters:
    - generator (nn.Module): The generator model.
    - discriminator (nn.Module): The discriminator model.
    - optimizer_g (torch.optim.Optimizer): Optimizer for the generator.
    - optimizer_d (torch.optim.Optimizer): Optimizer for the discriminator.
    - device (torch.device): Device to run computations on (CPU or GPU).
    - epochs (int): Number of training epochs.
    - batch_size (int): Size of each batch.
    - noise_dim (int): Dimension of the noise vector.
    - reshaped_data (numpy.ndarray): Real data samples.
    - lambda_gp (float): Gradient penalty coefficient.
    """
    reshaped_data_tensor = torch.tensor(reshaped_data, dtype=torch.float).to(device)

    num_batches = int(np.ceil(len(reshaped_data_tensor) / batch_size))
    losses = []

    for epoch in range(epochs):
        for batch_idx in range(num_batches):
            # Calculate start and end indices for the current batch
            start_idx = batch_idx * batch_size
            end_idx = min(start_idx + batch_size, len(reshaped_data_tensor))

            # Generate real data samples
            real_data_batch = reshaped_data_tensor[start_idx:end_idx]

            # Generate synthetic data samples
            
            noise = generate_noise(len(real_data_batch), noise_dim).to(device)
            fake_data = generator(noise)
            # === Train Discriminator ===
            optimizer_d.zero_grad()
            real_validity = discriminator(real_data_batch)
            fake_validity = discriminator(fake_data.detach())

            # Compute Wasserstein loss for discriminator
            d_loss_real = wasserstein_loss(torch.ones_like(real_validity), real_validity)
            d_loss_fake = wasserstein_loss(-torch.ones_like(fake_validity), fake_validity)

            # Gradient penalty

            gp =  gradient_penalty(discriminator, real_data_batch, fake_data)
            d_loss = d_loss_real + d_loss_fake + gp
            d_loss.backward()
            optimizer_d.step()

            # === Train Generator ===
            optimizer_g.zero_grad()
            # Generate fake data again for generator update
            fake_data = generator(noise)
            fake_validity = discriminator(fake_data)

            # Compute Wasserstein loss for generator and physics loss
            #g_loss=−E[D(fake samples)] the generator want D to give fake data max score , we want min problem so we add negative 
            g_loss = -wasserstein_loss(torch.ones_like(fake_validity), fake_validity) + physics_loss(fake_data)
            g_loss.backward()
            optimizer_g.step()

            # Optionally print the progress for each batch or save it
            losses.append({"epoch": epoch, "D Loss": d_loss.item(), "G Loss": g_loss.item()})
            print(f"Epoch {epoch+1}/{epochs}, Batch {batch_idx+1}/{num_batches}, D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")

    return losses

def train(generator, discriminator, optimizer_g, optimizer_d, device, epochs, noise_dim, lambda_gp, param_grid):
    """
    Train the GAN using Wasserstein loss with gradient penalty for the entire dataset at once.

    Parameters:
    - generator (nn.Module): The generator model.
    - discriminator (nn.Module): The discriminator model.
    - optimizer_g (torch.optim.Optimizer): Optimizer for the generator.
    - optimizer_d (torch.optim.Optimizer): Optimizer for the discriminator.
    - device (torch.device): Device to run computations on (CPU or GPU).
    - epochs (int): Number of training epochs.
    - noise_dim (int): Dimension of the noise vector.
    - reshaped_data (numpy.ndarray): Real data samples.
    - lambda_gp (float): Gradient penalty coefficient.
    """
    tensor_data = torch.tensor(reshaped_data, dtype=torch.float32)
    dataset = TensorDataset(tensor_data)
    data_loader = DataLoader(dataset, batch_size=33, shuffle=True)
    losses = []

    for epoch in range(epochs):
           for real_data_tensor, in data_loader:
            real_data_tensor = real_data_tensor.to(device)
            # Generate real data samples
            real_data_batch = real_data_tensor
            # Generate synthetic data samples
            fake_data = []
            for i in range(len(real_data_batch)):
                noise = generate_noise(1, noise_dim).to(device)
                fake_data_data = generator(noise)
                fake_data.append(fake_data_data)
                
            fake_data = torch.cat(fake_data, dim=0).to(device)
            
            # === Train Discriminator ===
            optimizer_d.zero_grad()
            real_validity = discriminator(real_data_batch)
            fake_validity = discriminator(fake_data.detach())
    
            # Compute Wasserstein loss for discriminator
            d_loss_real = wasserstein_loss(torch.ones_like(real_validity), real_validity)
            d_loss_fake = wasserstein_loss(-torch.ones_like(fake_validity), fake_validity)
    
            # Gradient penalty
    
            gp =  gradient_penalty(discriminator, real_data_batch, fake_data)
            d_loss = d_loss_real + d_loss_fake + gp
            d_loss.backward()
            optimizer_d.step()
    
            # === Train Generator ===
            optimizer_g.zero_grad()
            # Generate fake data again for generator update
            fake_data = []
            for i in range(len(real_data_batch)):
                noise = generate_noise(1, noise_dim).to(device)
                fake_data_data = generator(noise)
                fake_data.append(fake_data_data)
            fake_data = torch.cat(fake_data, dim=0).to(device)
            fake_validity = discriminator(fake_data)
    
            # Compute Wasserstein loss for generator and physics loss
            phy_factor = param_grid
            g_loss = -wasserstein_loss(torch.ones_like(fake_validity), fake_validity) + phy_factor * physics_loss(fake_data)
            g_loss.backward()
            optimizer_g.step()
    
            # Optionally print the progress for each batch or save it
            losses.append({"epoch": epoch, "D Loss": d_loss.item(), "G Loss": g_loss.item()})
            print(f"Epoch {epoch+1}/{epochs}, D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")

     
    d_losses = [loss['D Loss'] for loss in losses]  # Assuming this contains total D loss
    g_losses = [loss['G Loss'] for loss in losses]  # Assuming this contains G loss
    
    return generator, discriminator, np.mean(g_losses), losses


In [ ]:
# Parameters
epochs = 5000
batch_size = 33  # Set batch_size appropriately
noise_dim = 100  # Dimensionality of the input noise vector
num_features = 27

generator = Generator(noise_dim, num_features)
generator.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

discriminator = Discriminator(features_num)
optimizer_g, optimizer_d, device = setup_gan(generator, discriminator, lr=0.0002)

# Start training
# Grid search

best_loss = float('inf')
best_params = {}
phy_factor_param = [0.2] #[0.2, 0.5, 1, 2 ]
lambda_gp=10
for value in phy_factor_param:
    generator, discriminator, gloss, losses =train(generator, discriminator, optimizer_g, optimizer_d, device, epochs, noise_dim, lambda_gp, value)
    if gloss < best_loss:
        best_loss = gloss
        best_params = {"phy_factor_param":value, "generator_model": generator, "discriminator_model": discriminator, "gloss": gloss, "losses":losses}
        
print("Training completed.")


In [ ]:
# Assuming `generator` and `discriminator` are your model instances
generator = best_params['generator_model']
discriminator = best_params['discriminator_model']
losses = best_params['losses']

torch.save(generator.state_dict(), 'generator_model_phase1WGAN.pth')
torch.save(discriminator.state_dict(), 'discriminator_model_phase1WGAN.pth')


In [ ]:
best_params

In [ ]:
from torchviz import make_dot 

# Generate fake data
noise = generate_noise(len(real_data), noise_dim)
# Random noise
fake_data = generator(noise)

dot = make_dot(generator(noise), params=dict(generator.named_parameters()))
dot.render('generator_graph_WGAN', format='pdf')  # Saves the graph as a PNG file

In [ ]:
import matplotlib.pyplot as plt

# Assuming `losses` is a list of dictionaries containing 'epoch', 'D Loss', and 'G Loss'
# Extracting the data
epochs_ls = [i for i in range(1, epochs + 1)]  # Integer epochs
d_losses = [loss['D Loss'] for loss in losses]  # Assuming this contains total D loss
g_losses = [loss['G Loss'] for loss in losses]  # Assuming this contains G loss
print(np.mean(d_losses),"d_losses")
print(np.mean(g_losses),"g_losses")

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(epochs_ls, d_losses, label='Discriminator Loss')
plt.plot(epochs_ls, g_losses, label='Generator Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Losses Over Epochs')
plt.legend()
plt.savefig('Training_Losses_Over_Epochswgan.pdf')
plt.show()


In [ ]:
def generate_samples(generator, sample_count, noise_dim,seed=None):
    """
    Generate samples using the generator.

    Parameters:
    - generator (nn.Module): The generator model.
    - sample_count (int): Total number of samples to generate.
    - noise_dim (int): Dimension of the noise vector.
    - seed (int, optional): Random seed for reproducibility.

    Returns:
    - torch.Tensor: Generated samples tensor.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    batch_size = len(reshaped_data)
    all_samples = []

    # Calculate number of full batches
    num_full_batches = sample_count // batch_size

    for i in range(num_full_batches):
        # Generate noise for the batch
        noise = generate_noise(batch_size, noise_dim).to(device)
        
        with torch.no_grad():  # Disable gradient calculation for inference
            samples = generator(noise).cpu()  # Move samples to CPU for concatenation
        all_samples.append(samples)

    # Handle the remainder if sample_count isn't a perfect multiple of batch_size
    remainder = sample_count % batch_size
    if remainder > 0:
        noise = generate_noise(remainder, noise_dim, seed=seed).to(device)
        with torch.no_grad():  # Disable gradient calculation for inference
            samples = generator(noise).cpu()  # Move samples to CPU for concatenation

        all_samples.append(samples)

    # Concatenate all samples into a single tensor
    return torch.cat(all_samples, dim=0)


In [ ]:
# Generate samples
noise_dim = 100 # This should match the dimension used during training
sample_count = 1000

# Generate samples using a for loop
generated_samples = []
for i in range(sample_count):
    sample = generate_samples(generator, 1, noise_dim)
    generated_samples.append(sample)




In [ ]:

torch.onnx.export(generator,  generate_noise(1, noise_dim), "model.onnx")

In [ ]:
# Concatenate the generated samples
concatenated_tensor = torch.cat(generated_samples, dim=0)

# Convert to NumPy array (if you need to save it as a .npy file)
concatenated_array = concatenated_tensor.cpu().numpy()
4# Save to file
np.save("syntheticData_100Samplewgan.npy", concatenated_array)
concatenated_array.shape


In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting
plt.figure(figsize=(10, 6))
# Selecting the last column from each series
energy_col = concatenated_array[:, :, -1]
energy_sums = np.sum(energy_col, axis=1)
time_col = concatenated_array[:, :, -2]

# Summing across the rows (time steps)
time_sums = np.sum(time_col, axis=1)

plt.scatter(time_sums, energy_sums, alpha=0.6)
plt.title('Synthetic Paths - Distribution of Cycle Time and Energy')
plt.xlabel('Time (seconds)')
plt.ylabel('Energy')
plt.savefig('Synthetic Data - Distribution of Cycle Time and Energywgan.pdf')
plt.show()


In [ ]:

# Extract normalized time and energy into separate variables
denormalize_generated_data = denormalize_data(concatenated_array, scaler, features)
denormalize_generated_data.to_csv("denormalized_sythData_phase1_waed_wgan.csv")

time_denorm = denormalize_generated_data['Time']  # Assuming time is the second last column
energy_denorm = denormalize_generated_data['Energy_Consumped']  # Assuming energy is the last column

# Ensure the series length is a multiple of 5
n = 5
length = len(time_denorm)
trimmed_length = length - (length % n)
# Trim the series if necessary
trimmed_series = time_denorm[:trimmed_length]
# Reshape to groups of 5
reshaped_array = trimmed_series.to_numpy().reshape(-1, n)
# Sum each group of 5
time_denorm_sums = reshaped_array.sum(axis=1)
trimmed_series = energy_denorm[:trimmed_length]
# Reshape to groups of 5
reshaped_array = energy_denorm.to_numpy().reshape(-1, n)
# Sum each group of 5
energy_denorm_sums = reshaped_array.sum(axis=1)

# Plotting
plt.scatter(time_denorm_sums, energy_denorm_sums, color='blue')

# Set title and labels
plt.title('Synthetic Paths - Distribution of Cycle Time and Energy')
plt.xlabel('Time (seconds)')
plt.ylabel('Energy')

# Save and show the plot
plt.savefig('denormalize-Synthetic Datawgan.pdf')
plt.show()



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt 
sns.set_style('whitegrid')

last_col = real_data.iloc[:, 17]
# Group by each set of five rows and sum
# np.arange(len(last_col)) // 5 creates an integer index for each group of five
energy_real = last_col.groupby(np.arange(len(last_col)) // 5).sum()
last_col = real_data.iloc[:, 16]
time_real = last_col.groupby(np.arange(len(last_col)) // 5).sum()

plt.scatter(time_denorm_sums, energy_denorm_sums, color='blue',label='Synthetic Psths')
plt.scatter(time_real, energy_real, color='green',label='Real Paths')

plt.title('Synthetic & Real Paths- Distribution of Cycle time and Energy')
plt.xlabel('Time (seconds)')
plt.ylabel('Energy')
plt.legend()
plt.savefig('Synthetic & Real Paths- Distribution of Cycle time and Energy wgan.pdf')

plt.show()

In [ ]:
col_16 = time_denorm_sums.flatten()
plt.figure(figsize=(10, 6))
sns.histplot(col_16, kde=True, bins=10 , label = "Time")
plt.title('Synthetic Data - Distribution of Cycle time')
plt.xlabel('Time (seconds)')
plt.ylabel('Frequency')
plt.legend()
plt.savefig("denormalized_data- Sythetic Data - Distribution of Cycle time wgan.pdf")
plt.show()

In [ ]:


# Function to determine the Pareto front for minimizing both dimensions
def pareto_frontier_minimize_both(Xs, Ys):
    sorted_points = sorted([[Xs[i], Ys[i]] for i in range(len(Xs))], key=lambda point: (point[0], point[1]))
    pareto_front = [sorted_points[0]]
    for current in sorted_points[1:]:
        if current[1] < pareto_front[-1][1]:
            pareto_front.append(current)
    return [pair[0] for pair in pareto_front], [pair[1] for pair in pareto_front]

# Compute Pareto front
pareto_X, pareto_Y = pareto_frontier_minimize_both(time_denorm_sums, energy_denorm_sums)

# Plotting the points and Pareto front
plt.scatter(time_denorm_sums,energy_denorm_sums, color='blue', label='Sythetic Paths')  # Your original data points
plt.scatter(time_real, energy_real, color='green', label='Real Paths')
plt.plot(pareto_X, pareto_Y, color='red', linewidth=2.0, label='Pareto Front')  # Pareto front

plt.title('Synthetic & Real Paths - Distribution of Cycle Time and Energy')
plt.xlabel('Time (seconds)')
plt.ylabel('Energy')
plt.legend()
plt.savefig('Synthetic & Real Paths- Pareto Visulization wgan.pdf')
plt.show()

# Identifying the absolute optimal point (minimum energy and time)
min_energy = min(pareto_Y)
optimal_points = [(x, y) for x, y in zip(pareto_X, pareto_Y) if y == min_energy]
print("Optimal Points (Min Energy and then Min Time):", optimal_points)
print(pareto_X, pareto_Y)

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# Generate random data points for Time and Energy
np.random.seed(0)  # For reproducibility
time = time_denorm_sums
energy = energy_denorm_sums

# Function to identify Pareto optimal points
def identify_pareto(scores):
    # Initialize a boolean array to identify Pareto points
    is_pareto = np.ones(scores.shape[0], dtype=bool)
    for i in range(scores.shape[0]):
        for j in range(scores.shape[0]):
            if all(scores[j] <= scores[i]) and any(scores[j] < scores[i]):
                is_pareto[i] = False
                break
    return is_pareto

# Identify Pareto front
pareto_points = identify_pareto(np.c_[time, energy])
pareto_time = time[pareto_points]
pareto_energy = energy[pareto_points]

# Ensure there are Pareto points before proceeding

if pareto_time.size > 0 and pareto_energy.size > 0:
    # Select the best point based on the shortest Euclidean distance from the origin
    distances = np.sqrt(pareto_time**2 + pareto_energy**2)
    best_index = np.argmin(distances)
    best_time = pareto_time[best_index]
    best_energy = pareto_energy[best_index]

    # Plotting
    plt.figure(figsize=(10, 6))
    plt.scatter(time, energy, color='gray', label='Sythetic Paths')
    plt.scatter(time_real, energy_real, color='green', label='Real Paths')

    plt.scatter(pareto_time, pareto_energy, color='blue', label='Pareto Front')
    plt.scatter(best_time, best_energy, color='red', label='Best Point')
    plt.plot([0, best_time], [best_energy, best_energy], 'k--', lw=1)  # Horizontal line to Best Point
    plt.plot([best_time, best_time], [0, best_energy], 'k--', lw=1)  # Vertical line to Best Point
    plt.text(best_time, best_energy, ' Best Point', verticalalignment='bottom')
    plt.xlabel('Time')
    plt.ylabel('Energy')
    plt.title('Pareto Front with Best Selected Point')
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("No Pareto optimal points were identified.")
print(best_time,best_energy)

In [ ]:
import math


# Improvement calculation
def improvement_ratio(real, generated):
    time_improvement = (real[0] - generated[0]) / real[0] if real[0] != 0 else 0
    energy_improvement = (real[1] - generated[1]) / real[1] if real[1] != 0 else 0
    return time_improvement, energy_improvement

# Example usage:
real_point = (3.18, 58.5091)
generated_point = (best_time, best_energy)


# Get improvements
time_improvement, energy_improvement = improvement_ratio(real_point, generated_point)



print(f"Time Improvement Ratio: {time_improvement:.2%}")
print(f"Energy Improvement Ratio: {energy_improvement:.2%}")
generated_point


In [ ]:
indexes_phase1=[]
data = {
    'Time': denormalize_generated_data['Time'],   # Replace 50 with the actual size
    'Energy_Consumped': denormalize_generated_data['Energy_Consumped']
}
df_indexes = pd.DataFrame(data)

# Group by every five rows and sum
grouping_key = np.floor(np.arange(len(df_indexes)) / 5)

grouped = df_indexes.groupby(grouping_key).sum()
grouped['Index'] = grouped.index.astype(int) * 5  

for t, e in zip(pareto_time, pareto_energy):
    print(t,e)
    try:
        index = int(grouped[grouped['Time'] == t].index[0])
        index_2 = int(grouped[grouped['Energy_Consumped'] == e].index[0])
    except IndexError:
        index = None
        index_2 = None
    if (index ==index_2):
        indexes_phase1.append(index)

# Convert to a DataFrame
df_indexes = pd.DataFrame(indexes_phase1, columns=['path_index'])
df_indexes.drop_duplicates(inplace=True)
print(df_indexes)
# Save to a CSV file
df_indexes.to_csv('pathes_index_phase1.csv', index=False)
denormalize_generated_data

# Phase 2  , Take the fav/accurate points from the engineer then re-fine-tune the generator 

### Load Pre-trained GAN

In [ ]:
# Load the trained model weights
generator.load_state_dict(torch.load('generator_model_phase1WGAN.pth'))
discriminator.load_state_dict(torch.load('discriminator_model_phase1WGAN.pth'))

optimizer_g, optimizer_d, device = setup_gan(generator, discriminator, lr=0.0002)

### Define Human Feedback Mechanism
Define the reinforcement learning components needed for HFRL, such as the policy network (generator) and the value network.

Policy Network (Generator):


The policy network in RL terminology is the component that decides which actions to take. In the context of a GAN, the generator acts as the policy network because it generates new samples (actions) based on some input noise.
We reuse the pre-trained generator model for this purpose.

### Value Network:


The value network estimates the expected reward for a given state. In our context, it helps evaluate how good the generated samples are, based on the human feedback.
The value network is a separate neural network that takes the same input as the generator and outputs a single value representing the expected reward.

In [ ]:

def collect_human_feedback(samples):
    # Ensure samples are on GPU
    samples = samples.to(device)
    alpha = 0.5
    feedback =[]
    for sample in samples:
        #lower values are better
        rating = (sample[:,-2].sum() * alpha + sample[:,-1].sum() *(1-alpha))
        feedback.append(float(rating))
    
    return torch.tensor(feedback, dtype=torch.float).to(device)


### 3. Define RL Components

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Assuming `generator_ph1` is your policy network and already defined
policy_network = generator

# Define the value network in PyTorch
class ValueNetwork(nn.Module):
    def __init__(self, input_features):
        super(ValueNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.dense1 = nn.Linear(input_features, 128)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(128, 1)
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        return x

# Assume 'features' is defined elsewhere and provides the number of input features
input_dim = 5 * len(features)  # Update with the actual size of 'features'
value_network = ValueNetwork(input_dim).to(device)

optimizer_value = optim.Adam(value_network.parameters(), lr=0.0002)


### 4. Modify GAN Training Loop

Integrate the human feedback into the GAN training loop. Update the generator based on the human feedback using RL techniques.

In [ ]:
losses_ph2 = []
def train_ph2(generator, discriminator, optimizer_g, optimizer_d, device, epochs, noise_dim, hfrl_paths_normalized_3d, value_network, optimizer_value, lambda_gp=20):
    for epoch in range(epochs):
        real_data_batch = torch.from_numpy(hfrl_paths_normalized_3d).float().to(device)

        noise = generate_noise(len(real_data_batch), noise_dim).to(device)
        fake_data = generator(noise)

        # Train Discriminator
        optimizer_d.zero_grad()
        real_validity = discriminator(real_data_batch)
        fake_validity = discriminator(fake_data.detach())
        d_loss_real = wasserstein_loss(torch.ones_like(real_validity), real_validity)
        d_loss_fake = wasserstein_loss(-torch.ones_like(fake_validity), fake_validity)
        gp = gradient_penalty(discriminator, real_data_batch, fake_data)
        d_loss = d_loss_real + d_loss_fake + lambda_gp * gp
        d_loss.backward(retain_graph=True)
        optimizer_d.step()

        
        
        # Update Value Network
        feedback = collect_human_feedback(fake_data)
        optimizer_value.zero_grad()
        mse_loss = nn.MSELoss()
        if isinstance(fake_data, list):
            fake_data = torch.tensor(fake_data, dtype=torch.float).to(device)  # Convert list to tensor
        predicted_feedback = value_network(fake_data)
        feedback = feedback.view(predicted_feedback.shape)
        value_loss = mse_loss(feedback, predicted_feedback)
        value_loss.backward(retain_graph=True)
        optimizer_value.step()

         # Train Generator
        optimizer_g.zero_grad()
        fake_data = generator(noise)
        fake_validity = discriminator(fake_data)
        phy_factor = 0.5
        phy_loss = phy_factor * physics_loss(fake_data)
        g_loss = -wasserstein_loss(torch.ones_like(fake_validity), fake_validity)
        
        #lower values are better
        #modify the generator's loss to include this feedback negatively (since you typically minimize loss)
        val_factor = 1
        phy_factor = 0.5
        total_g_loss = g_loss + val_factor * value_loss.detach() + phy_factor * phy_loss  # where lambda is a weighting factor
        total_g_loss.backward()
        optimizer_g.step()
        
        losses_ph2.append({"epoch": epoch, "D Loss": d_loss.item(), "G Loss": total_g_loss.item(), "Value Loss": value_loss.item()})
        print(f"Epoch {epoch+1}/{epochs}, D Loss: {d_loss.item()}, G Loss: {total_g_loss.item()}, Value Loss: {value_loss.item()}")

    return losses_ph2


In [ ]:
#hfrl_paths = pd.read_csv("waeeeed paths after engineer review it.csv")
hfrl_paths = pd.read_csv("phase RLHF dataset/least_energy_time_paths.csv")

# Fix a typo in the 'point_type' column
real_data['point_type'] = hfrl_paths['point_type'].replace('approuch', 'approach')

# Map 'point_type' and 'Movement Type' to numeric values
points = {'pick': 1, 'approach': 2, 'start_point': 3, 'box1': 4, 'way_point': 5}
movement_types = {'MoveL': 1, 'MoveJ': 2, 'Movel': 1, 'Movej': 2}
hfrl_paths['point_type'] = hfrl_paths['point_type'].map(points)
hfrl_paths['Movement Type'] = hfrl_paths['Movement Type'].map(movement_types)

# Drop columns 'id' and 'row_index'
hfrl_paths.drop(['id'], axis=1, inplace=True)


hfrl_paths_normalized, scaler_ph2 = normalizaData(hfrl_paths)

#exclude the  Joint_W3 and keep sin , cos
hfrl_paths_normalized= hfrl_paths_normalized[features]
num_paths = len(hfrl_paths) // 5
batch_size = num_paths #all data 

path_id = np.repeat(np.arange(num_paths), 5)
hfrl_paths_normalized['path_id'] = path_id
grouped = hfrl_paths_normalized.groupby('path_id').apply(lambda x: x.values[:, :-1].reshape(1, 5,features_num))
hfrl_paths_normalized.drop(['path_id'], axis=1, inplace=True)

# Concatenate the groups back into a single numpy array
hfrl_paths_normalized_3d = np.concatenate(grouped.values, axis=0)

optimizer_g, optimizer_d, device = setup_gan(generator, discriminator, lr=0.0002)
losses_ph2 =  train_ph2(generator, discriminator, optimizer_g, optimizer_d, device, epochs, noise_dim, hfrl_paths_normalized_3d, value_network ,optimizer_value,  lambda_gp=20)


In [ ]:
# Extracting the data

#Epoch: 41, Batch: 34, D Loss: 0.7058187127113342, G Loss: 0.6761125326156616, Value Loss: 9.605087280273438

epochs_ls = [loss['epoch'] for loss in losses_ph2]
d_losses = [loss['D Loss'] for loss in losses_ph2]
g_losses = [loss['G Loss'] for loss in losses_ph2]
value_losses =  [loss['Value Loss'] for loss in losses_ph2]
d_losses_real = []
d_losses_fake = []

print(np.mean(d_losses),"d_losses")
print(np.mean(g_losses),"g_losses")
print(np.mean(value_losses),"reward")

# Plotting
plt.figure(figsize=(10, 5))
#d_loss_real
plt.plot(epochs_ls, d_losses, label='Discriminator Loss')

plt.plot(epochs_ls, g_losses, label='Generator Loss')
plt.plot(epochs_ls, value_losses, label='Reward Loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Losses Over Epochs')
plt.legend()
plt.savefig('Training Losses Over Epochs wgan.pdf')
plt.show()

In [ ]:
# Generate samples
noise_dim = 100  # This should match the dimension used during training
sample_count = 1000

# Generate samples using a for loop
samples_syth = []
for i in range(sample_count):
    sample = generate_samples(generator, 1, noise_dim)
    samples_syth.append(sample)
    
samples_syth = torch.cat(samples_syth, dim=0)

# Convert to NumPy array (if you need to save it as a .npy file)
samples_syth = samples_syth.cpu().numpy()

### Denormalize Dataset

In [ ]:
samples_syth_ph2 = denormalize_data(samples_syth, scaler_ph2,features)
samples_syth_ph2.head(2)

In [ ]:

# Extract normalized time and energy into separate variables
denormalize_generated_data = denormalize_data(concatenated_array, scaler, features)

time_denorm = samples_syth_ph2['Time']  # Assuming time is the second last column
energy_denorm = samples_syth_ph2['Energy_Consumped']  # Assuming energy is the last column

# Ensure the series length is a multiple of 5
n = 5
length = len(time_denorm)
trimmed_length = length - (length % n)
# Trim the series if necessary
trimmed_series = time_denorm[:trimmed_length]
# Reshape to groups of 5
reshaped_array = trimmed_series.to_numpy().reshape(-1, n)
# Sum each group of 5
time_denorm_sums = reshaped_array.sum(axis=1)
trimmed_series = energy_denorm[:trimmed_length]
# Reshape to groups of 5
reshaped_array = energy_denorm.to_numpy().reshape(-1, n)
# Sum each group of 5
energy_denorm_sums = reshaped_array.sum(axis=1)

# Plotting
plt.scatter(time_denorm_sums, energy_denorm_sums, color='blue')

# Set title and labels
plt.title('Synthetic Paths - Distribution of Cycle Time and Energy')
plt.xlabel('Time (seconds)')
plt.ylabel('Energy')

# Save and show the plot
plt.savefig('denormalize-Synthetic Data.pdf')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sample data 
time = samples_syth_ph2['Time']
energy = samples_syth_ph2['Energy_Consumped']

# Function to determine the Pareto front for minimizing both dimensions
def pareto_frontier_minimize_both(Xs, Ys):
    sorted_points = sorted([[Xs[i], Ys[i]] for i in range(len(Xs))], key=lambda point: (point[0], point[1]))
    pareto_front = [sorted_points[0]]
    for current in sorted_points[1:]:
        if current[1] < pareto_front[-1][1]:
            pareto_front.append(current)
    return [pair[0] for pair in pareto_front], [pair[1] for pair in pareto_front]

# Compute Pareto front
pareto_X, pareto_Y = pareto_frontier_minimize_both(time, energy)

# Plotting the points and Pareto front
plt.scatter(time, energy, color='blue', label='Sythetic Paths')  # Your original data points
plt.scatter(time_real, energy_real, color='green', label='Real Paths')
plt.plot(pareto_X, pareto_Y, color='red', linewidth=2.0, label='Pareto Front')  # Pareto front

plt.title('Synthetic & Real Paths - Distribution of Cycle Time and Energy')
plt.xlabel('Time (seconds)')
plt.ylabel('Energy')
plt.legend()
plt.savefig('Synthetic & Real Paths- Pareto Visulization wgan.pdf')
plt.show()

# Identifying the a-bsolute optimal point (minimum energy and time)
min_energy = min(pareto_Y)
optimal_points = [(x, y) for x, y in zip(pareto_X, pareto_Y) if y == min_energy]
print("Optimal Points (Min Energy and then Min  Time):", optimal_points)
print(pareto_X, pareto_Y)

In [ ]:
time = time_denorm_sums
energy =energy_denorm_sums



# Function to identify Pareto optimal points
def identify_pareto(scores):
    # Initialize a boolean array to identify Pareto points
    is_pareto = np.ones(scores.shape[0], dtype=bool)
    for i in range(scores.shape[0]):
        for j in range(scores.shape[0]):
            if all(scores[j] <= scores[i]) and any(scores[j] < scores[i]):
                is_pareto[i] = False
                break
    return is_pareto

# Identify Pareto front
pareto_points = identify_pareto(np.c_[time, energy])
pareto_time = time[pareto_points]
pareto_energy = energy[pareto_points]

# Ensure there are Pareto points before proceeding

if pareto_time.size > 0 and pareto_energy.size > 0:
    # Select the best point based on the shortest Euclidean distance from the origin
    distances = np.sqrt(pareto_time**2 + pareto_energy**2)
    best_index = np.argmin(distances)
    best_time = pareto_time[best_index]
    best_energy = pareto_energy[best_index]

    # Plotting
    plt.figure(figsize=(10, 6))
    plt.scatter(time, energy, color='gray', label='Sythetic Paths')
    plt.scatter(time_real, energy_real, color='green', label='Real Paths')

    plt.scatter(pareto_time, pareto_energy, color='blue', label='Pareto Front')
    plt.scatter(best_time, best_energy, color='red', label='Best Point')
    plt.plot([0, best_time], [best_energy, best_energy], 'k--', lw=1)  # Horizontal line to Best Point
    plt.plot([best_time, best_time], [0, best_energy], 'k--', lw=1)  # Vertical line to Best Point
    plt.text(best_time, best_energy, ' Best Point', verticalalignment='bottom')
    plt.xlabel('Time')
    plt.ylabel('Energy')
    plt.title('Pareto Front with Best Selected Point')
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("No Pareto optimal points were identified.")
print(best_time,best_energy)

In [ ]:
import math


# Improvement calculation
def improvement_ratio(real, generated):
    time_improvement = (real[0] - generated[0]) / real[0] if real[0] != 0 else 0
    energy_improvement = (real[1] - generated[1]) / real[1] if real[1] != 0 else 0
    return time_improvement, energy_improvement

# Example usage:
real_point = (3.18, 58.5091)
generated_point = (best_time, best_energy)


# Get improvements
time_improvement, energy_improvement = improvement_ratio(real_point, generated_point)



print(f"Time Improvement Ratio: {time_improvement:.2%}")
print(f"Energy Improvement Ratio: {energy_improvement:.2%}")

In [ ]:
!pip install onnx onnx-tf
